In [25]:
import pandas as pd
import eeg
import datetime
import scipy.fftpack
import numpy as np
import matplotlib.pyplot as plt


files = eeg.extractBundledEEG("../../scripts/data/")
files.addMeta("../../scripts/data/eeg-restingstate/events.csv","timeofday")
files.prune()
# files.mergeTagsWithRegex("[cC]lose","Eyes Closed")
files.categories

dataAgg = dict()

for x in ["morning","evening"]:
    for y in files.extractByTags(x):
        if x not in dataAgg: dataAgg[x] = {}
        dataAgg[x][y] = eeg.load_session(files.extractById(y))["avg_power_by_band"]

1674926196 Meta available but no associated recordings


Simple Aggregated Trials

In [28]:
morning = pd.DataFrame.from_dict(dataAgg["morning"])
morning["1674239692"].std()

2.636825054280294